# Tutorial
This tutorial demonstrates how to use MOGT functions with a demo dataset (SCZ as an example). Once you are familiar with SCZ’s workflow, please replace the demo data with your own data to begin your analysis. 

## How to prepare input data

We recommend getting started with SCZ using the provided demo dataset. When you want to apply SCZ to your own multi-omics dataset, please refer to the following tutorials to learn how to prepare input data.

Overall, the input data consists of two parts: the graph, constructed from SNP-SNP interaction and the node feature including DE, EPI, and gene expression in five brain regions（Parietal Lobe, Frontal Lobe, Temporal Lobe, Cerebellum, and Occipital Lobe）in adolescents and adults.

If you are unfamiliar with MOGT, you may start with our data used in the paper to save your time. For SCZ, the input data as well as the label information are uploaded [here](https://github.com/NBStarry/CGMega/tree/main/data). If you start with this data, you can skip the _step 1_ about _How to prepare input data_.

> The labels should be collected yourself if you choose analyze your own data.

### Import default params and set constants

In [1]:
import utils
from config import config_load
from data_preprocess_cv import *

configs = config_load.get()
disease = configs["disease"]
# Set a GPU with enough memory. Set to 'cpu' if you don't have a GPU, but it will be very slow.
GPU_ID = 4
configs["gpu"] = f"cuda:{GPU_ID}"
RANDOM_SEED = configs['random_seed']
CV_FOLDS = configs['cv_folds']
GENE_LIST = utils.get_gene_list(rename=True,disease = disease)
GENE_LIST[:10]

/home/lijf/miniconda3/envs/MOGT/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Gene Name,gene_id
0,ATAD3A,ENSG00000197785
1,SSU72,ENSG00000160075
2,SLC35E2B,ENSG00000189339
3,NADK,ENSG00000008130
4,GNB1,ENSG00000078369
5,PRKCZ,ENSG00000067606
6,SKI,ENSG00000157933
7,MORN1,ENSG00000116151
8,RER1,ENSG00000157916
9,PEX10,ENSG00000157911


### load omics data
Download demo data from [Google Drive](https://drive.google.com/drive/folders/1w4GGIZRUNevTkWUSsLKPNS5AliHeOvi8?usp=sharing), put it into DATA_DIR, and run:

In [2]:
node_feat, pos = get_node_feat(disease=disease)
feat_cols = ["de","brain_cp","brain_gz","neuron_count","oligo_count","micro_count","adolescence_parietal.lobe","adolescence_frontal.lobe","adolescence_temporal.lobe","adolescence_cerebellum","adolescence_occipital.lobe","adulthood_parietal.lobe","adulthood_frontal.lobe","adulthood_temporal.lobe","adulthood_cerebellum","adulthood_occipital.lobe"]
feat_df = pd.DataFrame(data=node_feat, columns=feat_cols)
feat_df = pd.concat([GENE_LIST, feat_df], axis=1)
feat_df[:10]

data.dim:(3829, 16)


,Gene Name,gene_id,de,brain_cp,brain_gz,neuron_count,oligo_count,micro_count,adolescence_parietal.lobe,adolescence_frontal.lobe,adolescence_temporal.lobe,adolescence_cerebellum,adolescence_occipital.lobe,adulthood_parietal.lobe,adulthood_frontal.lobe,adulthood_temporal.lobe,adulthood_cerebellum,adulthood_occipital.lobe
0,ATAD3A,ENSG00000197785,0.55910,27.0,34.0,0.0,0.0,0.0,3.108212,3.499470,2.986322,2.874243,3.100803,2.702168,2.877700,2.781966,3.006098,2.696311
1,SSU72,ENSG00000160075,0.11310,19.0,22.0,0.0,0.0,0.0,17.607839,17.853936,16.868172,20.382898,16.619251,18.799567,18.319362,18.245816,21.977522,18.646992
2,SLC35E2B,ENSG00000189339,0.60410,25.0,30.0,0.0,0.0,0.0,6.456090,6.462289,6.212757,6.696293,5.911461,8.606304,8.729064,7.582789,8.896779,9.627115
3,NADK,ENSG00000008130,0.81800,21.0,17.0,3.0,3.0,4.0,4.124521,4.508349,4.245148,3.896298,4.052343,3.891285,3.955317,3.869454,4.393209,3.687674
4,GNB1,ENSG00000078369,0.70940,26.0,20.0,3.0,5.0,3.0,125.859036,132.714089,120.597878,114.350219,114.216024,146.239985,142.608107,124.627841,117.265747,142.000320
5,PRKCZ,ENSG00000067606,0.92230,18.0,12.0,0.0,0.0,0.0,25.745606,29.105656,25.391222,34.795389,23.775744,30.950651,30.451210,28.287727,42.881957,31.037185
6,SKI,ENSG00000157933,0.20800,17.0,17.0,2.0,0.0,6.0,15.148673,15.094114,13.749281,57.878832,14.397054,13.648012,13.972990,12.997937,58.751702,15.281962
7,MORN1,ENSG00000116151,0.80020,10.0,11.0,0.0,0.0,0.0,1.175714,1.082000,0.992462,6.412815,1.314842,1.292641,1.156712,0.978280,7.694690,1.363268
8,RER1,ENSG00000157916,0.60650,9.0,8.0,7.0,1.0,10.0,10.112455,11.315149,10.519379,9.798119,10.024427,10.301282,10.577038,11.256921,12.127360,9.945186
9,PEX10,ENSG00000157911,0.05513,12.0,13.0,0.0,0.0,17.0,13.877252,14.387175,14.283532,10.596385,15.116163,10.898929,11.331722,13.198850,12.062927,10.838501


### SNP-SNP interaction construction
Then, we read the SNP-SNP interaction data ([Google Drive](https://drive.google.com/drive/folders/1w4GGIZRUNevTkWUSsLKPNS5AliHeOvi8?usp=sharing)) and transform it into a graph through the following commands:

In [3]:
snp_mat = get_snp_mat(disease=disease)
snp_mat[20:30, 20:30]

Loading SNP matrix from data//SCZ/SCZNetwork.csv ......


array([[2.9198e-04, 7.3044e-05, 7.3028e-05, 7.3003e-05, 7.3181e-05,
        7.2980e-05, 7.2962e-05, 7.2959e-05, 7.3263e-05, 7.3011e-05],
       [5.2597e-05, 2.7163e-04, 5.2573e-05, 5.2838e-05, 5.2630e-05,
        5.2476e-05, 5.2667e-05, 5.2449e-05, 5.2872e-05, 5.2507e-05],
       [6.5537e-05, 6.5521e-05, 2.8466e-04, 6.5498e-05, 6.5573e-05,
        6.5381e-05, 6.5368e-05, 6.5429e-05, 6.5457e-05, 6.5592e-05],
       [4.3688e-05, 4.3912e-05, 4.3677e-05, 2.6285e-04, 4.3727e-05,
        4.3720e-05, 4.3606e-05, 4.3605e-05, 4.3726e-05, 4.3689e-05],
       [2.2951e-04, 2.2923e-04, 2.2916e-04, 2.2916e-04, 4.4743e-04,
        2.2903e-04, 2.2928e-04, 2.2944e-04, 2.2961e-04, 2.2889e-04],
       [8.9168e-05, 8.9040e-05, 8.9014e-05, 8.9262e-05, 8.9225e-05,
        3.0802e-04, 8.9361e-05, 8.9536e-05, 8.9060e-05, 8.9134e-05],
       [1.7045e-04, 1.7087e-04, 1.7016e-04, 1.7023e-04, 1.7079e-04,
        1.7086e-04, 3.8959e-04, 1.7060e-04, 1.7038e-04, 1.7083e-04],
       [1.3450e-04, 1.3428e-04, 1.3441e-0

### Load gene labels

In [4]:
node_lab, labeled_idx = get_label(disease=disease )
labeled_lab = [node_lab[i][1] for i in labeled_idx]
print(f"Positive samples: {sum(node_lab)[1]}  Negative samples: {sum(node_lab)[0]}  Unlabeled samples: {len(node_lab) - len(labeled_idx)}")
print(node_lab)

Positive samples: 483.0  Negative samples: 1968.0  Unlabeled samples: 1378
[[0. 0.]
 [0. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]


### Build PyG Data instance

In [5]:
from data_preprocess_cv import build_pyg_data
edge_threshhold = configs["edge_threshhold"]
random = configs["random"]
data = build_pyg_data(GENE_LIST,node_feat, node_lab, snp_mat, pos,edge_threshhold,random)
data

Number of edges: 146523, Dimensionality of edge: 1,
Nubmer of nodes: 3829


Data(x=[3829, 16], edge_index=[2, 146523], edge_attr=[146523, 1], y=[3829, 2], pos=[3829], edge_dim=1, gene_name=     Gene Name          gene_id
0       ATAD3A  ENSG00000197785
1        SSU72  ENSG00000160075
2     SLC35E2B  ENSG00000189339
3         NADK  ENSG00000008130
4         GNB1  ENSG00000078369
...        ...              ...
3824    GABRA3  ENSG00000011677
3825    ZNF275  ENSG00000063587
3826       BGN  ENSG00000182492
3827    PLXNB3  ENSG00000198753
3828        F8  ENSG00000185010

[3829 rows x 2 columns])

### Do train-test split

25% for test set, 75% for a 10-fold train-valid spilt.

In [6]:
from data_preprocess_cv import split_data
train_idx_list,valid_idx_list,test_idx = split_data(CV_FOLDS,labeled_idx,labeled_lab)

### Create CancerDataset and Save

In [7]:
import pickle
cv_dataset = create_cv_dataset(train_idx_list.copy(), valid_idx_list.copy(), test_idx.copy(), data=data)
print(cv_dataset[0])
dataset_dir = "data/" + configs["disease"] + "/"+ configs["disease"]+ "_dataset.pkl"
print(f'Finished! Saving dataset to {dataset_dir} ......')
with open(dataset_dir, 'wb') as f:
    pickle.dump(cv_dataset, f)

Data(x=[3829, 16], edge_index=[2, 146523], edge_attr=[146523, 1], y=[3829, 2], pos=[3829], edge_dim=1, gene_name=     Gene Name          gene_id
0       ATAD3A  ENSG00000197785
1        SSU72  ENSG00000160075
2     SLC35E2B  ENSG00000189339
3         NADK  ENSG00000008130
4         GNB1  ENSG00000078369
...        ...              ...
3824    GABRA3  ENSG00000011677
3825    ZNF275  ENSG00000063587
3826       BGN  ENSG00000182492
3827    PLXNB3  ENSG00000198753
3828        F8  ENSG00000185010

[3829 rows x 2 columns], train_mask=[3829, 5], valid_mask=[3829, 5], test_mask=[3829], unlabeled_mask=[3829], num_classes=2)
Finished! Saving dataset to data/SCZ/SCZ_dataset.pkl ......


### Load data and Train model <div id="load-data-and-train-model"></div>
| Item       | Details          |
| ---------- | ---------------- |
| System     | Ubuntu 20.04.1 LTS |
| RAM Memory | 256G               |
| GPU Memory | NVIDIA GeForce RTX, 24G     |

```note
The above table reports our computing details during CGMega development and IS NOT our computing requirement.

If your computer does not satisfy the above, you may try to lower down the memory used during model training by reduce the sampling parameters, the batch size or so on. 

We are going to test CGMega under more scenarios like with different models of GPU or memory limits to update this table.

```

A 5-fold training would cost 41m on our computer.

In [8]:
from main import get_training_modules, train_model, predict, calculate_metrics, pred_to_df,test
from data_preprocess_cv import scale_data

# To load dataset from above, set
# configs['stable'] = False
# To use a stable version in the repo, set
configs['stable'] = True
# CV_FOLDS = 1
# The two version should be the same by a consistent process.

# set to 'cuda' to use GPU.
configs['device'] = 'cuda'

# Set log_name and logfile to suit your needs, and you will see training details in logfile.
# configs['log_name'] = f"{get_cell_line(configs['data_dir'])[1:]}_{configs['ppi']}"
configs['log_name'] = configs["disease"]
configs['logfile'] = os.path.join(configs["log_dir"], configs["log_name"] + ".txt")

dataset = get_data(configs,disease)


def calculate_confidence_interval(scores):
    mean = np.mean(scores)
    std = np.std(scores)
    n = len(scores)
    z = 1.96  # 95% 置信水平的Z值
    bound = (z * std / np.sqrt(n))
    return bound

sum_auprc, sum_auc, sum_acc, sum_f1, sum_tp, train_result = [], [], [], [], [], []
for i in range(CV_FOLDS):
    head_info = True if i == 0 else False 
    configs['fold'] = i
    data = dataset
    data = scale_data(data,i)
    modules = get_training_modules(configs, data)
    auprc, auc, acc, f1, tp, new_ckpt,cutoff = train_model(modules, configs, configs['log_name'], i, head_info,)
    y_score, y_pred, y_true, y_index, genes = predict(modules['model'], modules['test_loader_list'], configs, new_ckpt)
    acc, cf_matrix, auprc, f1, auc, test_cutoff = calculate_metrics(y_true, y_pred, y_score)
     #save test result
    # test_true,test_pred,test_score,genes = test(modules['model'], modules['test_loader_list'],configs['device'],cutoff,i)
    y = pd.DataFrame({'y_true': y_true, 'y_score': y_score,"y_pred":y_pred,"cutoff":cutoff})
    result = pd.concat([genes,y], axis=1)
    result.to_csv(configs["out_dir"]+"/"+disease +"/"+disease+"_"+"test"+"_"+str(i)+".csv",index_label=False)


    tp = cf_matrix[1, 1]
    sum_auprc.append(auprc)
    sum_auc.append(auc)
    sum_acc.append(acc)
    sum_f1.append(f1)
    sum_tp.append(tp)
    with open(configs['logfile'], 'a') as f:
        print("Test AUPRC:{:.4f}, AUROC:{:.4f}, ACC:{:.4f}, F1:{:.4f}, TP:{:.1f},cutoff:{:.4f}"
            .format(auprc, auc, acc, f1, tp,test_cutoff), file=f, flush=True)
    train_result =  pred_to_df(i, train_result, y_index, y_true, y_score)
auc_ci = calculate_confidence_interval(sum_auc)
auprc_ci = calculate_confidence_interval(sum_auprc)
avg_auc = sum(sum_auc) / len(sum_auc)
avg_Auprc = sum(sum_auprc) / len(sum_auprc)
avg_fi = sum(sum_f1) / len(sum_f1)
with open(configs['logfile'], 'a') as f:
    print(f"{CV_FOLDS}-folds AUPRC:{avg_Auprc:.4f}+{auprc_ci}, AUROC:{avg_auc:.4f}+{auc_ci}, F1:{avg_fi:.4f}",
            file=f, flush=True)



Loading dataset from: data/SCZ_core/SCZ_core_dataset.pkl ......
start loading model
start load data
Start Training
Epoch: 4, Focal loss: 0.0375, contrastive loss: 0.0168, Train loss: 0.0377, Train Acc: 0.8393, Train Auprc: 0.5614, Train Auroc: 0.7404
Epoch: 4, Valid Acc: 0.8240, Valid Auprc: 0.5343, Valid TP: 11, Valid F1: 0.2418, Valid Auroc: 0.7216
Epoch: 9, Focal loss: 0.0288, contrastive loss: 0.0130, Train loss: 0.0290, Train Acc: 0.8463, Train Auprc: 0.5977, Train Auroc: 0.7622
Epoch: 9, Valid Acc: 0.8495, Valid Auprc: 0.5880, Valid TP: 27, Valid F1: 0.4779, Valid Auroc: 0.7948
Epoch: 14, Focal loss: 0.0274, contrastive loss: 0.0088, Train loss: 0.0275, Train Acc: 0.8616, Train Auprc: 0.6387, Train Auroc: 0.8096
Epoch: 14, Valid Acc: 0.8495, Valid Auprc: 0.6092, Valid TP: 25, Valid F1: 0.4587, Valid Auroc: 0.8190
Epoch: 19, Focal loss: 0.0265, contrastive loss: 0.0024, Train loss: 0.0265, Train Acc: 0.8642, Train Auprc: 0.6627, Train Auroc: 0.8275
Epoch: 19, Valid Acc: 0.8367, Va

In [9]:
#scz_core
from main import predict_all
cutoff = 0.3171
predict_all(configs,cutoff)

Loading dataset from: data/SCZ_core/SCZ_core_dataset.pkl ......
checkpoints:['3_SCZ_core_0.8235.pkl', '2_SCZ_core_0.8545.pkl', '4_SCZ_core_0.8848.pkl', '1_SCZ_core_0.8672.pkl', '0_SCZ_core_0.8593.pkl'] 
start loading model
start load data
Loading model from ./outs/SCZ_core/4_SCZ_core_0.8848.pkl ......
Loading model from ./outs/SCZ_core/4_SCZ_core_0.8848.pkl ......
0.3171
候选基因的长度：1271
